## Weighted stats
- This notebook is made in the Programming for Data Analytics lecture on weighted descriptive statistics.
- We are looking at popultions in each county of ireland


In [1]:
import pandas as pd

In [2]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
9789,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-149d-13a3-e055-000000000001,Cavan County Council,Number,12
9790,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-14a4-13a3-e055-000000000001,Donegal County Council,Number,31
9791,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-1495-13a3-e055-000000000001,Monaghan County Council,Number,7


ok I notice that this has all the data for each gender I want to get rid of that

In [3]:
df = df[df["Sex"] != "Female"]
df = df[df["Sex"] != "Male"]
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
3261,FY006AC01,Population,2022,2022,-,Both sexes,650,100 years and over,2ae19629-149d-13a3-e055-000000000001,Cavan County Council,Number,18
3262,FY006AC01,Population,2022,2022,-,Both sexes,650,100 years and over,2ae19629-14a4-13a3-e055-000000000001,Donegal County Council,Number,33
3263,FY006AC01,Population,2022,2022,-,Both sexes,650,100 years and over,2ae19629-1495-13a3-e055-000000000001,Monaghan County Council,Number,8


Now I can do the code that is the prep for analysis.py  
I just noticed that there were more columns so I need to get their names

In [4]:
headers = df.columns.tolist()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [5]:
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','Sex','C02076V03371','C03789V04537','UNIT']
df.drop(columns=drop_col_list, inplace=True)
df = df[df["Single Year of Age"] != "All ages"]
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)

df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE']=df['VALUE'].astype('int64')
#print (df.head(3))
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3232 entries, 32 to 3263
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Single Year of Age       3232 non-null   int64 
 1   Administrative Counties  3232 non-null   object
 2   VALUE                    3232 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 101.0+ KB


In [6]:
df_anal = pd.pivot_table(df, 'VALUE',"Single Year of Age","Administrative Counties")
print (df_anal.head(3))
# write out the entire file to local machine
df_anal.to_csv("population_for_analysis.csv")

Administrative Counties  Carlow County Council  Cavan County Council  \
Single Year of Age                                                     
0                                          699                  1006   
1                                          649                  1004   
2                                          689                  1061   

Administrative Counties  Clare County Council  Cork City Council  \
Single Year of Age                                                 
0                                        1377               2283   
1                                        1360               2284   
2                                        1473               2306   

Administrative Counties  Cork County Council  Donegal County Council  \
Single Year of Age                                                     
0                                       4190                    1797   
1                                       4115                    1881   
2         

## Now we can do weighted descriptive statistics

In [7]:
headers = list(df_anal.columns)
district = headers[0]
district

'Carlow County Council'

Weighted mean is sum(age*population at age) / sum (populations at age)

In [11]:
number_people = df_anal[district].sum()
number_people


61968

In [12]:
df_anal

Administrative Counties,Carlow County Council,Cavan County Council,Clare County Council,Cork City Council,Cork County Council,Donegal County Council,Dublin City Council,Dún Laoghaire Rathdown County Council,Fingal County Council,Galway City Council,...,Monaghan County Council,Offaly County Council,Roscommon County Council,Sligo County Council,South Dublin County Council,Tipperary County Council,Waterford City & County Council,Westmeath County Council,Wexford County Council,Wicklow County Council
Single Year of Age,,,,,,,,,,,,,,,,,,,,,
0,699,1006,1377,2283,4190,1797,6213,2457,4009,792,...,868,935,748,743,3544,1932,1340,1129,1792,1710
1,649,1004,1360,2284,4115,1881,5699,2437,3894,757,...,821,937,764,766,3673,1808,1266,1158,1724,1682
2,689,1061,1473,2306,4290,1971,5830,2489,4308,792,...,870,945,893,764,3734,1836,1329,1108,1835,1796
3,754,1085,1468,2294,4437,2074,5580,2613,4384,799,...,900,1009,859,825,3856,2048,1476,1148,1864,1862
4,745,1047,1505,2243,4620,2089,5624,2658,4422,830,...,873,1007,833,820,3870,2072,1390,1208,1941,1897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,11,23,40,54,84,48,200,95,48,19,...,17,19,28,21,41,41,26,24,42,38
97,11,13,31,39,65,27,115,57,51,12,...,9,16,13,22,38,33,20,23,37,25
98,7,8,15,23,47,15,81,45,21,15,...,11,8,7,12,17,21,17,11,22,16


In [13]:
cumages = df_anal[district].mul(df_anal.index, axis=0).sum()
cumages

2371825

In [14]:
weighted_mean = cumages/number_people
weighted_mean

38.27499677252776

#### Or you can use numpy

In [15]:
import numpy as np
w_mean = np.average(df_anal.index, weights=df_anal[district])
w_mean

38.27499677252776

### weighted median

    create a series of the cumlative sums and find the index of the middle value

In [ ]:
cumsum = df_anal[district].cumsum()
cumsum

In [ ]:
# sum of all the number of people
cutoff = df_anal[district].sum()/2
cutoff

In [ ]:
df_anal[district][cumsum>=cutoff].index[0]

## Weighted standard deviation

The is the same formula as the normal standard deviation, just with weights applied to the differenced to the weight mean

In [ ]:
# incorrect way
df_anal[district].std()

In [ ]:
# incorrect
df_anal[district].mean()

We need the weighted std

In [ ]:
import numpy as np
w_mean = np.average(df_anal.index, weights=df_anal[district])
w_mean

In [ ]:
w_variance = np.average((df_anal.index - w_mean)**2, weights= df_anal[district])
w_variance

In [ ]:
w_std = np.sqrt(w_variance)
w_std